# Purpose of this Notbook

- understand how to download an aoi from http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/README
    
    

# get and cat the readme

In [77]:
#! wget http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/README -O rangeland_cover_v2_0.txt

--2020-12-11 21:13:18--  http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/README
Resolving rangeland.ntsg.umt.edu (rangeland.ntsg.umt.edu)... 150.131.193.3
Connecting to rangeland.ntsg.umt.edu (rangeland.ntsg.umt.edu)|150.131.193.3|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2616 (2.6K)
Saving to: ‘rangeland_cover_v2_0.txt’

rangeland_cover_v2_ 100%[===================>]   2.55K  --.-KB/s    in 0s      

2020-12-11 21:13:19 (303 MB/s) - ‘rangeland_cover_v2_0.txt’ saved [2616/2616]



In [2]:
!cat rangeland_cover_v2_0.txt

rangeland cover v2.0
These data represent rangeland cover estimates described in Allred et al.
(2020).

Band 1 - annual forb and grass
Band 2 - bare ground
Band 3 - litter
Band 4 - perennial forb and grass
Band 5 - shrub
Band 6 - tree
Band 7 - annual forb and grass uncertainty
Band 8 - bare ground uncertainty
Band 9 - litter uncertainty
Band 10 - perennial forb and grass uncertainty
Band 11 - shrub uncertainty
Band 12 - tree uncertainty

No Data value = 65535
Uncertainty values are scaled by 100

Uncertainty values represent a standard deviation of predictions. As such, they
may be standardized by the mean when comparing across groups. Appropriate
actions should be taken when the mean is less than one.

Although these data were produced across a broad region, they are primarily
intended for rangeland ecosystems. Cover estimates may not be suitable in other
ecosystems, e.g., forests, agricultural lands.

Coordinate reference system
Data are in WGS84 Geographic Coordinate System (EPSG:43

# Download tip
============
To download a specific location, use the GDAL virtual file system. For example,
the following gdal_translate command will retrieve a small section of Montana
(see the gdal_translate documentation for more information):

gdal_translate -co compress=lzw -co tiled=yes -co bigtiff=yes \
/vsicurl/http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/vegetation-cover-v2-2019.tif \
-projwin -108 48 -107 47 out.tif

# Guess at Neal's AOI

In [1]:
! cat AOI/tonyaoi.geojson| python -m json.tool

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            -121.11328124999999,
                            38.634036452919226
                        ],
                        [
                            -115.04882812499999,
                            38.634036452919226
                        ],
                        [
                            -115.04882812499999,
                            45.874712248904764
                        ],
                        [
                            -121.11328124999999,
                            45.874712248904764
                        ],
                        [
                            -121.11328124999999,
                            38.634036452919226
                        ]
                

## No Data value = 65535
- Uncertainty values are scaled by 100

# Display it on a map for tony

In [2]:
my_aoi_file = 'AOI/tonyaoi.geojson'
from notebookLib.nb_display import Ldisplay
ld = Ldisplay()
ld.map_geojson(my_aoi_file)

creating Ldisplay class


In [6]:
# how much disk did tony give me? - these things are big!
! df -h .

Filesystem      Size  Used Avail Use% Mounted on
/dev/nvme0n1p1  233G   24G  210G  10% /


In [7]:
# ok

In [8]:
import geojson

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = f"({box[0][0]} {box[1][0]}, {box[0][1]} {box[1][1]})"
     return ret

# obviously you need to parse your json here
poly=geojson.Polygon([[(2.38, 57.322), (23.194, -20.28), (-120.43, 19.15), (2.38, 57.322)]])
line = bbox(list(geojson.utils.coords(poly)))
print(line)

(-120.43 -20.28, 23.194 57.322)


In [9]:
import json

In [75]:
with open(my_aoi_file) as f:
  geo_data = json.load(f)

In [76]:
geo_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-121.11328124999999, 38.634036452919226],
      [-115.04882812499999, 38.634036452919226],
      [-115.04882812499999, 45.874712248904764],
      [-121.11328124999999, 45.874712248904764],
      [-121.11328124999999, 38.634036452919226]]]}}]}

In [32]:
geo_data['features'][0]['geometry']

{'type': 'Polygon',
 'coordinates': [[[-121.11328124999999, 38.634036452919226],
   [-115.04882812499999, 38.634036452919226],
   [-115.04882812499999, 45.874712248904764],
   [-121.11328124999999, 45.874712248904764],
   [-121.11328124999999, 38.634036452919226]]]}

In [33]:
line = bbox(list(geojson.utils.coords(geo_data['features'][0]['geometry'])))
print(line)

(-121.11328124999999 38.634036452919226, -115.04882812499999 45.874712248904764)


In [34]:
! gdal_translate -co compress=lzw -co tiled=yes -co bigtiff=yes \
/vsicurl/http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/vegetation-cover-v2-2019.tif \
-projwin -121.11328124999999 38.634036452919226, -115.04882812499999 45.874712248904764 oregon_nevada.tif

Input file size is 116045, 86299
ERROR 1: Error: Computed -srcwin 13570 38723 22503 -26868 has negative width and/or height.


In [35]:
#! gdal_translate -co compress=lzw -co tiled=yes -co bigtiff=yes \
/vsicurl/http://rangeland.ntsg.umt.edu/data/rap/rap-vegetation-cover/v2/vegetation-cover-v2-2019.tif \
-projwin -121.11328124999999 45.874712248904764, -115.04882812499999 38.634036452919226 TIF/oregon_nevada.tif

Input file size is 116045, 86299
0...10...20...30...40...50...60...70...80...90...100 - done.


# that took about 30 minutes - and it gave me progress

- the whole file was 130 Gigabytes
- my part of CONUS was only 8.6 Gig - still big for a file

In [38]:
!ls -lh

total 8.6G
-rw-r--r-- 1 jupyter-harmony jupyter-harmony 3.1K Dec 11 18:36 00-readme.ipynb
-rw-r--r-- 1 jupyter-harmony jupyter-harmony  20K Dec 11 19:45 01-download-trials-annual-herbacious.ipynb
-rw-r--r-- 1 jupyter-harmony jupyter-harmony 1.9K Dec 11 18:50 Readme.md
-rw-r--r-- 1 jupyter-harmony jupyter-harmony 8.6G Dec 11 19:46 oregon_nevada.tif
-rw-r--r-- 1 jupyter-harmony jupyter-harmony 2.6K Jun 12 06:00 rangeland_cover_v2_0.txt
-rw-r--r-- 1 jupyter-harmony jupyter-harmony  329 Dec 11 18:56 tonyaoi.geojson


In [39]:
!ls -lh or*

-rw-r--r-- 1 jupyter-harmony jupyter-harmony 8.6G Dec 11 19:46 oregon_nevada.tif


In [4]:
! rio info TIF/oregon_nevada.tif | python -m json.tool

{
    "blockxsize": 256,
    "blockysize": 256,
    "bounds": [
        -121.11329205626103,
        38.633935255656674,
        -115.04885540469856,
        45.874715771773666
    ],
    "colorinterp": [
        "gray",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined",
        "undefined"
    ],
    "compress": "lzw",
    "count": 12,
    "crs": "EPSG:4326",
    "descriptions": [
        null,
        null,
        null,
        null,
        null,
        null,
        null,
        null,
        null,
        null,
        null,
        null
    ],
    "driver": "GTiff",
    "dtype": "uint16",
    "height": 26868,
    "indexes": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12
    ],
    "interleave": "pixel",
    "lnglat": [
        -118.0

In [3]:
!gdalinfo TIF/oregon_nevada.tif

Driver: GTiff/GeoTIFF
Files: TIF/oregon_nevada.tif
Size is 22503, 26868
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (-121.113292056261031,45.874715771773666)
Pixel Size = (0.000269494585236,-0.000269494585236)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (-121.1132921,  45.8747158) (121d 6'47.85"W, 45d52'28.98"N)
Lower Left  (-121.1132921,  38.6339353) (121d 6'47.85"W, 38d38' 2.17"N)
Upper Right (-115.0488554,  45.8747158) (115d 2'55.88"W, 45d52'28.98"N)
Lower Right (-115.0488554,  38.6339353) (115d 2'55.88"W, 38d38' 2.17"N)
Center      (-118.0810737,  42.2543255) (118d 4'51.87"W, 42d15'15.57"N)
Band 1 Block=256x256 Type=UInt16, ColorInterp=Gray
  NoData Value=65535

In [45]:
!rio --help

Usage: rio [OPTIONS] COMMAND [ARGS]...

  Rasterio command line interface.

Options:
  -v, --verbose           Increase verbosity.
  -q, --quiet             Decrease verbosity.
  --aws-profile TEXT      Select a profile from the AWS credentials file
  --aws-no-sign-requests  Make requests anonymously
  --aws-requester-pays    Requester pays data transfer costs
  --version               Show the version and exit.
  --gdal-version
  --help                  Show this message and exit.

Commands:
  blocks     Write dataset blocks as GeoJSON features.
  bounds     Write bounding boxes to stdout as GeoJSON.
  calc       Raster data calculator.
  clip       Clip a raster to given bounds.
  convert    Copy and convert raster dataset.
  edit-info  Edit dataset metadata.
  env        Print information about the Rasterio environment.
  gcps       Print ground control points as GeoJSON.
  info       Print information about a data file.
  insp       Open a data file and start an interpreter.
  mask

In [46]:
! rio clip --help

Usage: rio clip [OPTIONS] INPUT OUTPUT

  Clips a raster using projected or geographic bounds.

    $ rio clip input.tif output.tif --bounds xmin ymin xmax ymax
    $ rio clip input.tif output.tif --like template.tif

  The values of --bounds are presumed to be from the coordinate reference
  system of the input dataset unless the --geographic option is used, in
  which case the values may be longitude and latitude bounds. Either JSON,
  for example "[west, south, east, north]", or plain text "west south east
  north" representations of a bounding box are acceptable.

  If using --like, bounds will automatically be transformed to match the
  coordinate reference system of the input.

  It can also be combined to read bounds of a feature dataset using Fiona:

    $ rio clip input.tif output.tif --bounds $(fio info features.shp --bounds)

Options:
  -o, --output PATH               Path to output file (optional alternative to
                                  a positional arg).

  --bound

In [53]:
! rio clip TIF/oregon_nevada.tif TIF/little_oregon.tif --bounds '-121.0 38.7 -120.0 39.7'

In [54]:
!ls -lh *.tif

-rw-r--r-- 1 jupyter-harmony jupyter-harmony 220M Dec 11 20:38 little_oregon.tif
-rw-r--r-- 1 jupyter-harmony jupyter-harmony 8.6G Dec 11 19:46 oregon_nevada.tif


In [5]:
!gdalinfo TIF/little_oregon.tif

Driver: GTiff/GeoTIFF
Files: TIF/little_oregon.tif
Size is 3711, 3711
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (-121.000000000000000,39.700000000000003)
Pixel Size = (0.000269494585236,-0.000269494585236)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (-121.0000000,  39.7000000) (121d 0' 0.00"W, 39d42' 0.00"N)
Lower Left  (-121.0000000,  38.6999056) (121d 0' 0.00"W, 38d41'59.66"N)
Upper Right (-119.9999056,  39.7000000) (119d59'59.66"W, 39d42' 0.00"N)
Lower Right (-119.9999056,  38.6999056) (119d59'59.66"W, 38d41'59.66"N)
Center      (-120.4999528,  39.1999528) (120d29'59.83"W, 39d11'59.83"N)
Band 1 Block=256x256 Type=UInt16, ColorInterp=Gray
  NoData Value=65535
B

In [57]:
!rio --help

Usage: rio [OPTIONS] COMMAND [ARGS]...

  Rasterio command line interface.

Options:
  -v, --verbose           Increase verbosity.
  -q, --quiet             Decrease verbosity.
  --aws-profile TEXT      Select a profile from the AWS credentials file
  --aws-no-sign-requests  Make requests anonymously
  --aws-requester-pays    Requester pays data transfer costs
  --version               Show the version and exit.
  --gdal-version
  --help                  Show this message and exit.

Commands:
  blocks     Write dataset blocks as GeoJSON features.
  bounds     Write bounding boxes to stdout as GeoJSON.
  calc       Raster data calculator.
  clip       Clip a raster to given bounds.
  convert    Copy and convert raster dataset.
  edit-info  Edit dataset metadata.
  env        Print information about the Rasterio environment.
  gcps       Print ground control points as GeoJSON.
  info       Print information about a data file.
  insp       Open a data file and start an interpreter.
  mask

In [6]:
!rio bounds TIF/little_oregon.tif

{"bbox": [-121.0, 38.69990559418974, -119.99990559418974, 39.7], "geometry": {"coordinates": [[[-121.0, 38.69990559418974], [-119.99990559418974, 38.69990559418974], [-119.99990559418974, 39.7], [-121.0, 39.7], [-121.0, 38.69990559418974]]], "type": "Polygon"}, "properties": {"filename": "little_oregon.tif", "id": "0", "title": "TIF/little_oregon.tif"}, "type": "Feature"}


In [63]:
# turns out little oregon is actually little_calif
!rio bounds TIF/little_oregon.tif >AOI/little_calif.geojson

In [64]:
my_aoi_file = 'AOI/little_calif.geojson'
from notebookLib.nb_display import Ldisplay
ld = Ldisplay()
ld.map_geojson(my_aoi_file)

creating Ldisplay class


ValueError: The FeatureCollection needs to contain a 'features' property

In [69]:
with open(my_aoi_file) as f:
  geo_data2 = json.load(f)

In [71]:
geo_data

{'bbox': [-121.0, 38.69990559418974, -119.99990559418974, 39.7],
 'geometry': {'coordinates': [[[-121.0, 38.69990559418974],
    [-119.99990559418974, 38.69990559418974],
    [-119.99990559418974, 39.7],
    [-121.0, 39.7],
    [-121.0, 38.69990559418974]]],
  'type': 'Polygon'},
 'properties': {'filename': 'little_oregon.tif',
  'id': '0',
  'title': 'little_oregon.tif'},
 'type': 'Feature'}

In [78]:
geo_data2

{'bbox': [-121.0, 38.69990559418974, -119.99990559418974, 39.7],
 'geometry': {'coordinates': [[[-121.0, 38.69990559418974],
    [-119.99990559418974, 38.69990559418974],
    [-119.99990559418974, 39.7],
    [-121.0, 39.7],
    [-121.0, 38.69990559418974]]],
  'type': 'Polygon'},
 'properties': {'filename': 'little_oregon.tif',
  'id': '0',
  'title': 'little_oregon.tif'},
 'type': 'Feature'}

In [7]:
!ls

00-geojson-map.ipynb
00-readme.ipynb
01-download-trials-annual-herbacious.ipynb
AOI
Characterizing_Land_Surface_Phenology_and_Exotic_A.pdf
Qotom
Readme.md
TIF
rangeland_cover_v2_0.txt


In [80]:
!gdalinfo AOI/little_oregon.tif

Driver: GTiff/GeoTIFF
Files: little_oregon.tif
Size is 3711, 3711
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (-121.000000000000000,39.700000000000003)
Pixel Size = (0.000269494585236,-0.000269494585236)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  COMPRESSION=LZW
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (-121.0000000,  39.7000000) (121d 0' 0.00"W, 39d42' 0.00"N)
Lower Left  (-121.0000000,  38.6999056) (121d 0' 0.00"W, 38d41'59.66"N)
Upper Right (-119.9999056,  39.7000000) (119d59'59.66"W, 39d42' 0.00"N)
Lower Right (-119.9999056,  38.6999056) (119d59'59.66"W, 38d41'59.66"N)
Center      (-120.4999528,  39.1999528) (120d29'59.83"W, 39d11'59.83"N)
Band 1 Block=256x256 Type=UInt16, ColorInterp=Gray
  NoData Value=65535
Band 